In [2]:
#import the required packages

import matplotlib
import matplotlib.pyplot as plt
import matplotlib.dates as md
import matplotlib.pylab as pylab
import quandl
import pandas as pd
import copy
import numpy as np
import scipy as sp
import pandas_datareader.data as web
import datetime
from time import time

#Deep learn
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler, StandardScaler
import sklearn.linear_model
import sklearn.metrics
import statsmodels.api as sm
from keras.layers import Input, Dense
from keras.models import Model, load_model
from keras import regularizers
from collections import defaultdict

#layout
import seaborn as sns
%matplotlib inline
matplotlib.rcParams[ 'figure.figsize' ] = ( 18, 7 )

Using TensorFlow backend.


In [1]:
#Function to retrieve data
def GetData(fileName):
    return pd.read_csv(fileName, header=0,usecols=['Date','Adj Close'], parse_dates=True, index_col='Date').astype('float32')

In [24]:
##should i add the other assets back
#???


#

data_ivv = GetData('Asset_Dataset/IVV.csv')
%store data_ivv

data_lqd = GetData('Asset_Dataset/LQD.csv')
%store data_lqd

data_shy = GetData('Asset_Dataset/SHY.csv')
%store data_shy

data_vnq = GetData('Asset_Dataset/VNQ.csv')
%store data_vnq

data_gld = GetData('Asset_Dataset/GLD.csv')
%store data_gld

data_efa = GetData('Asset_Dataset/EFA.csv')
%store data_efa

data_eem = GetData('Asset_Dataset/EEM.csv')
%store data_eem

data_emb = GetData('Asset_Dataset/EMB.csv')
%store data_emb

asset_table = pd.concat([data_ivv,data_lqd,data_shy,data_vnq,data_gld,data_efa,data_eem,data_emb],axis=1)
asset_table.columns = ['IVV close','LQD close','SHY close','VNQ close','GLD close','EFA close','EEM close','EMB close']

Stored 'data_ivv' (DataFrame)
Stored 'data_lqd' (DataFrame)
Stored 'data_shy' (DataFrame)
Stored 'data_vnq' (DataFrame)
Stored 'data_gld' (DataFrame)
Stored 'data_efa' (DataFrame)
Stored 'data_eem' (DataFrame)
Stored 'data_emb' (DataFrame)


In [18]:
#net change
#def net_change(data):

data_ivv_net = data_ivv.diff().dropna()
data_lqd_net = data_lqd.diff().dropna()
data_shy_net = data_shy.diff().dropna()
data_vnq_net = data_vnq.diff().dropna()
data_gld_net = data_gld.diff().dropna()
data_efa_net = data_efa.diff().dropna()
data_eem_net = data_eem.diff().dropna()
data_emb_net = data_emb.diff().dropna()



In [36]:
### Auto Encoding ###
    #1)Train an auto encoder model which compresses market information
    #2)Rank stocks by 2-normal difference -> select the subset as the portfolio
    #3)Linear + RELU as activation functions (p7 of Heaton et al)
    #4)W = (W1, W2), L2 regularization

asset = defaultdict(defaultdict)    

#5 neurons
encoding_dimension = 5
#number of features
num_asset = len(asset_table)

#connect all layers
# see 'Stacked Auto-Encoders' in paper
input_layer = Input(shape=(num_asset, ))
encoded = Dense(encoding_dimension, activation='relu', kernel_regularizer=regularizers.l2(0.01))(input_layer)
decoded = Dense(num_asset, activation= 'linear', kernel_regularizer=regularizers.l2(0.01))(encoded)
    
# construct and compile AE model
autoencoder = Model(input_layer, decoded)
autoencoder.compile(optimizer='sgd', loss='mean_squared_error')
    

In [37]:
# train autoencoder
#data_t = asset['calibrate']['net']
autoencoder.fit(asset_table, asset_table, shuffle=False, epochs=500, batch_size = 10)
autoencoder.save('model/retrack_autoencoder.csv')

# test/reconstruct market information matrix
reconstruct = autoencoder.predict(data_t)    


ValueError: Error when checking input: expected input_6 to have shape (262,) but got array with shape (8,)